# Dataset Exploration (PlantVillage)
This notebook is used to **explore the PlantVillage dataset** after preparation.
We will:
1. List all available classes.
2. Display a few example images to inspect quality.
3. Count how many images exist per class (class balance).
4. Decide whether data augmentation is needed.

In [ ]:
from src import config
import random
import matplotlib.pyplot as plt
from PIL import Image

DATA_PROCESSED_DIR = config.DATA_PROCESSED_DIR
PLANTVILLAGE_DIR = DATA_PROCESSED_DIR / "PlantVillage"
TRAIN_DIR = PLANTVILLAGE_DIR / "train"
VAL_DIR = PLANTVILLAGE_DIR / "val"

assert TRAIN_DIR.exists(), f"Train folder not found at: {TRAIN_DIR}. Please run 00_prepare_data.ipynb first."
print(f"Train dir: {TRAIN_DIR.resolve()}")

## Step 1: List all classes
We list all subfolders in `data/processed/train`.
Each folder represents a separate class label.

In [ ]:
classes = sorted([p.name for p in TRAIN_DIR.iterdir() if p.is_dir()])
print(f"Detected {len(classes)} classes:")
for c in classes:
    print(" -", c)

## Step 2: Display Sample Images
Let’s visualize a few samples per class to check:
* image readability
* labeling consistency
* potential corrupted or empty files

In [ ]:
N_PER_CLASS = 3

MAX_CLASSES_TO_SHOW = 5
classes_to_show = classes[:MAX_CLASSES_TO_SHOW]

fig, axes = plt.subplots(len(classes_to_show), N_PER_CLASS, figsize=(N_PER_CLASS * 3, len(classes_to_show) * 3))

if len(classes_to_show) == 1:
    axes = [axes]

for i, cls in enumerate(classes_to_show):
    cls_dir = TRAIN_DIR / cls
    images = list(cls_dir.iterdir())
    images = [p for p in images if p.is_file()]
    random.shuffle(images)
    images = images[:N_PER_CLASS]

    for j in range(N_PER_CLASS):
        ax = axes[i][j] if len(classes_to_show) > 1 else axes[j]
        if j < len(images):
            try:
                img = Image.open(images[j])
            except Exception as e:
                print(f"Bad image: {images[j]} ({e})")
                ax.axis("off")
                continue
            ax.imshow(img)
            ax.set_title(cls, fontsize=8)
            ax.axis("off")
        else:
            ax.axis("off")

plt.tight_layout()
plt.show()

## Step 3: Count Images per Class

We count how many image files exist for each class in the training directory.
This helps identify imbalanced classes.

In [ ]:
IMAGE_EXTS = (".jpg", ".jpeg", ".png", ".bmp")

class_counts = {}
for cls in classes:
    cls_dir = TRAIN_DIR / cls
    count = sum(1 for p in cls_dir.iterdir() if p.is_file() and p.suffix.lower() in IMAGE_EXTS)
    class_counts[cls] = count

for cls, count in sorted(class_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"{cls:45s}: {count}")

### Visualize Class Balance

A simple bar chart gives an overview of the dataset distribution.

In [ ]:
plt.figure(figsize=(10, 4))
plt.bar(class_counts.keys(), class_counts.values())
plt.xticks(rotation=90)
plt.title("Number of images per class (train)")
plt.tight_layout()
plt.show()

## Step 3b: Group classes by health status
We want to see how many images are "healthy" vs "not_healthy".

In [ ]:
health_counts = {
    "healthy": 0,
    "not_healthy": 0
}

for cls, count in class_counts.items():
    if "healthy" in cls.lower():
        health_counts["healthy"] += count
    else:
        health_counts["not_healthy"] += count

print("\nImages per health group:")
for k, v in health_counts.items():
    print(f"{k:12s}: {v}")

plt.figure(figsize=(4, 3))
plt.bar(health_counts.keys(), health_counts.values())
plt.title("Images per health group (train)")
plt.tight_layout()
plt.show()

## Step 3c: Display sample images per health group
Just to visualize the difference.

In [ ]:
def show_samples(base_dir, label_keyword, n=6):
    """
    Display random sample images from classes containing (or excluding) the keyword.

    Args:
        base_dir (Path): Base directory containing class subfolders.
        label_keyword (str): Keyword to filter classes (e.g., "healthy" or "not_healthy").
        n (int): Number of sample images to display.
    """
    if label_keyword == "not_healthy":
        matching_classes = [d for d in base_dir.iterdir() if d.is_dir() and "healthy" not in d.name.lower()]
    else:
        matching_classes = [d for d in base_dir.iterdir() if d.is_dir() and label_keyword in d.name.lower()]

    sample_paths = []
    for cls_dir in matching_classes:
        images = list(cls_dir.glob("*.jpg")) + list(cls_dir.glob("*.png"))
        if not images:
            continue
        sample_paths.extend(random.sample(images, min(len(images), max(1, n // len(matching_classes) or 1))))

    if not sample_paths:
        print(f"No images found for {label_keyword}.")
        return

    n_cols = 3
    n_rows = (len(sample_paths) + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 3, n_rows * 3))
    axes = axes.flatten()

    for ax, path in zip(axes, sample_paths):
        img = Image.open(path)
        ax.imshow(img)
        ax.set_title(path.parent.name, fontsize=8)
        ax.axis("off")

    for ax in axes[len(sample_paths):]:
        ax.axis("off")

    fig.suptitle(f"Examples of {label_keyword} leaves", fontsize=12)
    plt.tight_layout()
    plt.show()

show_samples(TRAIN_DIR, "healthy")
show_samples(TRAIN_DIR, "not_healthy")

## Step 4: Decide on Data Augmentation Needs
* Take the largest class as reference.
* Any class with less than half that size may benefit from augmentation.

In [ ]:
max_count = max(class_counts.values())
threshold = max_count / 2

underrepresented = [cls for cls, cnt in class_counts.items() if cnt < threshold]

print(f"Largest class: {max_count} images")
print(f"Threshold for 'low': {threshold:.0f} images\n")

if underrepresented:
    print("Classes that may benefit from data augmentation:")
    for cls in underrepresented:
        print(f" - {cls} ({class_counts[cls]} images)")
else:
    print("Dataset appears fairly balanced. You can still apply global augmentation to reduce overfitting.")

### Check Validation Folder
We can also check the number of validation images per class, just to ensure the split ratio looks consistent.

In [ ]:
if VAL_DIR.exists():
    val_counts = {}
    for cls in classes:
        cls_dir = VAL_DIR / cls
        if not cls_dir.exists():
            continue
        count = sum(1 for p in cls_dir.iterdir() if p.is_file() and p.suffix.lower() in IMAGE_EXTS)
        val_counts[cls] = count

    print("\nValidation images per class:")
    for cls, count in sorted(val_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"{cls:45s}: {count}")